사용자 쿼리 → WeatherAgent
   ↓
스케줄 에이전트 처리 (시간/장소/활동 추출) 
   ↓
기본 정보 수집 (시간/위치/활동)
   ↓
결과 파싱 및 검증
   ↓
날씨 정보 생성 (코멘트 체인 )
   ↓
최종 응답 반환


In [2]:
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
import datetime
from datetime import timezone, timedelta
from typing import TypedDict, List
from pydantic import BaseModel, Field
import json
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import pickle
import os

class GraphState(TypedDict):
    time: str
    location: str
    activity: str

class Schedule(BaseModel):
    time: str = Field(..., description="일정 시간")
    location: str = Field(..., description="일정 장소")
    activity: str = Field(..., description="활동 내용")

class AllSchedules(BaseModel):
    schedules: List[Schedule] = Field(..., description="일정 목록")
    
    class Config:
        json_encoders = {
            datetime: lambda v: v.isoformat()
        }

# Comment Template
comment_template = """You are an AI assistant that provides detailed weather information based on the user's schedule and activities.

Given a schedule in the following JSON format:
{input}


Response Format:
1. Basic Weather:
- Temperature/Feels Like
- Precipitation Probability
- Overall Weather

2. Detailed Information:
- Hourly Highlights
- Weather Impact on Activities

3.Activity Advice:
-Essentials/Precautions
-Alternative Suggestions (if necessary)

Always provide specific numbers and details, even if you need to make educated estimates.
Please respond in Korean."""

@tool
def get_current_time(tool_input: str = "") -> str:
    """
    Gets the current time.
    Returns the time in 'YYYY-MM-DD HH:MM:SS' format.
    If tool_input contains "내일", returns tomorrow's time.
    """
    current = datetime.datetime.now()
    if isinstance(tool_input, dict):  # dict 형태로 들어오는 경우 처리
        tool_input = tool_input.get('tool_input', '')
        
    if "내일" in str(tool_input):  # 문자열로 변환하여 검사
        current = current + timedelta(days=1)
    elif "모레" in str(tool_input):
        current = current + timedelta(days=2)
    return current.strftime('%Y-%m-%d %H:%M:%S')

@tool
def get_current_location(tool_input: str = "") -> str:
    """
    Gets the current location of the user.
    Returns the location name in Korean (e.g., '서울시 구로구').
    """
    print("get_current_location called!")  # 디버깅용
    
    #처음 그래프스테이트는 서울시 구로구로 설정
    # 쿼리나 캘린더에서 장소 정보를 가져오는 것으로 대체 가능
    
    return "서울시 구로구"

@tool
def get_nearest_activity(tool_input: str = "") -> str:
    """
    Gets the nearest upcoming activity from the user's calendar.
    Returns the activity as a string in Korean.
    If no upcoming activities are found, search for past activities. If it is still not found, returns "None".
    """
    try:
        # OAuth 2.0 인증 범위 설정
        SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
        
        creds = None
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                creds = pickle.load(token)
                
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)
                
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)

        service = build('calendar', 'v3', credentials=creds)
        
        # 현재 시간을 UTC로 가져오기 (timezone-aware)
        now = datetime.datetime.now(datetime.UTC).isoformat()

        events_result = service.events().list(
            calendarId='primary',
            timeMin=now,
            maxResults=1,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        
        events = events_result.get('items', [])
        if not events:
            return "None"
            
        print(f"Found calendar event: {events[0]['summary']}")  # 디버깅용
        return events[0]['summary']
        
    except Exception as e:
        print(f"Calendar access error: {str(e)}")
        return "None"

class WeatherAgent:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)
        
        # Tools list with decorated functions
        self.schedule_tools = [
            get_current_time,
            get_current_location,
            get_nearest_activity
        ]
        
        # Schedule processing components
        self.schedule_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an AI assistant that understands the user's schedule by time, location, and activity.
Identify the user's schedule (activity, time and location) from the user's query and output it in JSON format.

IMPORTANT: You must ensure proper time handling:
1. For relative time expressions, you MUST:
   - "내일": Use `get_current_time` tool with "내일" parameter
   - "모레": Use `get_current_time` tool with "모레" parameter
   - "이따가": Add 3 hours to current time
   - "밤에": Set to 20:00:00 on specified day
   - Use `get_current_time` tool for time resolution

2. For time expressions:
   - Format must be `YYYY-MM-DD HH:MM:SS`
   - You MUST use the `get_current_time` tool with appropriate parameters
   - If only date is mentioned without time, set time to 12:00:00
   - For unspecified times, pass relevant keywords to the tool

3. `activity`
   - Must be in Korean
   - Extract from query (e.g., 달리기, 산책, 미팅)
   - Use `get_nearest_activity` tool if no activity mentioned

4. `location` 
   - Must be specific Korean location
   - Use `get_current_location` tool if none specified

Always respond in this exact format:
{{
    "schedules": [
        {{
            "time": "YYYY-MM-DD HH:MM:SS",
            "location": "location_in_Korean",
            "activity": "activity_in_Korean"
        }}
    ]
}}

Example of using get_current_time tool for "내일":
- Input: "내일 날씨 어때?"
- Action: Call get_current_time with "내일" parameter
- Use returned time in schedule

{agent_scratchpad}"""),
            ("human", "{input}")
        ])
        
        self.schedule_parser = PydanticOutputParser(pydantic_object=AllSchedules)
        
        # Comment processing components
        self.comment_prompt = ChatPromptTemplate.from_template(comment_template)
        self.comment_chain = self.comment_prompt | self.llm
        
        # Create schedule agent
        self.schedule_agent = self._create_schedule_agent()

    def _create_schedule_agent(self):
        agent = create_tool_calling_agent(
            self.llm,
            self.schedule_tools,
            self.schedule_prompt
        )
        
        return AgentExecutor(
            agent=agent,
            tools=self.schedule_tools,
            verbose=False,
            max_iterations=3,
            early_stopping_method="force",
            handle_parsing_errors=True
        )

    def process_query(self, query: str) -> dict:
        try:
            # 쿼리에서 시간 키워드 확인 및 시간 가져오기
            time_param = "내일" if "내일" in query else ("모레" if "모레" in query else "")
                    
            try:
                # 일정 추출
                schedule_result = self.schedule_agent.invoke(
                    {"input": query},
                    config={"callbacks": None}
                )
                
                # 기본 정보 가져오기 (schedule_result 실패 시를 대비해 여기로 이동)
                current_time = get_current_time.invoke(time_param)
                current_location = get_current_location.invoke("")
                calendar_activity = get_nearest_activity.invoke("")
                
                # 결과 파싱
                if isinstance(schedule_result.get('output'), str):
                    schedule_json = json.loads(schedule_result['output'])
                else:
                    schedule_json = schedule_result.get('output', {})
                    
                # 활동이 None이거나 없는 경우 캘린더 체크
                if not schedule_json.get("schedules"):
                    schedule_json = {"schedules": []}
                
                if not schedule_json["schedules"]:
                    schedule_json["schedules"].append({
                        "time": current_time,
                        "location": current_location,
                        "activity": calendar_activity if calendar_activity != "None" else "None"
                    })
                else:
                    if schedule_json["schedules"][0].get("activity") in [None, "None", "날씨 확인"]:
                        if calendar_activity != "None":
                            schedule_json["schedules"][0]["activity"] = calendar_activity
                    
                    # 시간이 없거나 잘못된 경우 current_time 사용
                    if not schedule_json["schedules"][0].get("time") or \
                       schedule_json["schedules"][0].get("time") == "Not_mentioned":
                        schedule_json["schedules"][0]["time"] = current_time
                
                # 스케줄 파싱
                schedules = self.schedule_parser.parse(json.dumps(schedule_json))
                
                # 날씨 정보 생성
                comment_input = json.dumps(json.loads(schedules.json()), ensure_ascii=False)
                comment_result = self.comment_chain.invoke({
                    "input": comment_input
                })
                
                return {
                    'schedule': json.loads(schedules.json()),
                    'comment': comment_result.content if hasattr(comment_result, 'content') else str(comment_result)
                }
                
            except Exception as e:
                print(f"Schedule processing error: {str(e)}")
                raise
                
        except Exception as e:
            print(f"Final error handler: {str(e)}")
            
            # 기본 정보 가져오기 (오류 발생 시)
            try:
                current_time = get_current_time.invoke(time_param)
                current_location = get_current_location.invoke("")
                calendar_activity = get_nearest_activity.invoke("")
            except:
                # 완전한 fallback
                current = datetime.datetime.now()
                if "내일" in query:
                    current = current + timedelta(days=1)
                elif "모레" in query:
                    current = current + timedelta(days=2)
                current_time = current.strftime('%Y-%m-%d %H:%M:%S')
                current_location = "서울시 구로구"
                calendar_activity = "None"
            
            # 기본 응답 생성 시에도 시간 키워드 처리 유지
            default_schedule = {
                "schedules": [{
                    "time": current_time,
                    "location": current_location,
                    "activity": calendar_activity if calendar_activity != "None" else ("산책" if "산책" in query else "None")
                }]
            }
            
            # 날씨 정보 생성 시도
            try:
                comment_input = json.dumps(default_schedule, ensure_ascii=False)
                comment_result = self.comment_chain.invoke({
                    "input": comment_input
                })
                weather_comment = comment_result.content if hasattr(comment_result, 'content') else str(comment_result)
            except:
                # 날씨 정보 생성 실패 시 기본 응답
                weather_comment = f"{current_time}의 날씨 정보:\n- 기온: 최저 12도, 최고 22도\n- 강수확률: 10%\n- 날씨: 대체로 맑음\n- 미세먼지: 보통"
            
            return {
                'schedule': default_schedule,
                'comment': weather_comment
            }

def format_schedule(schedule_dict):
    """Format schedule information for display"""
    try:
        schedules = schedule_dict.get('schedules', [])
        if not schedules:
            return "분석된 일정이 없습니다."
            
        formatted = []
        for schedule in schedules:
            time_str = schedule.get('time', 'Not_mentioned')
            if time_str != 'Not_mentioned':
                time_str = time_str if isinstance(time_str, str) else time_str.isoformat()
            time_str = '시간 미지정' if time_str == 'Not_mentioned' else time_str
            
            location_str = '장소 미지정' if schedule.get('location') == 'Not_mentioned' else schedule.get('location')
            activity_str = '활동 미지정' if schedule.get('activity') == 'None' else schedule.get('activity')
            
            formatted.append(f"""
시간: {time_str}
장소: {location_str}
활동: {activity_str}
""".strip())
        
        return '\n\n'.join(formatted)
    except Exception as e:
        return f"일정 포맷팅 중 오류 발생: {str(e)}"

def main():
    weather_agent = WeatherAgent()
    
    test_queries = [
        "내일 날씨가 어떨까?",
        "오늘 남산에서 저녁 9시에 달리기할건데 날씨가 어떨까?",
        "오늘 저녁 9시에 달리기할건데 날씨가 어떨까?",
        "밤에 산책할건데 날씨가 괜찮을까?",
        "이따가 밤에 산책할건데 날씨가 괜찮을까?",
        "오늘 날씨가 어때?"
    ]
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n{'='*50}")
        print(f"테스트 케이스 #{i}")
        print(f"사용자 질문: {query}")
        print('-'*50)
        
        result = weather_agent.process_query(query)
        
        print("\n◆ 분석된 일정:")
        print(format_schedule(result['schedule']))
        print("\n◆ 날씨 정보 및 조언:")
        print(result['comment'].strip())

if __name__ == "__main__":
    main()


테스트 케이스 #1
사용자 질문: 내일 날씨가 어떨까?
--------------------------------------------------
Schedule processing error: 1 validation error for get_current_time
tool_input
  Input should be a valid string [type=string_type, input_value={'tool_input': '내일'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
Final error handler: 1 validation error for get_current_time
tool_input
  Input should be a valid string [type=string_type, input_value={'tool_input': '내일'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
get_current_location called!
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=9457781659-qbbr0aeb71sv4d13t8ht9enf5oaoho4u.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53349%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=kVsORM3KloOORwVOulPCEMM85oby9g&access_type=offline
Found c